In [2]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

 ········


In [3]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

 ········


In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1721930974, 'localtime': '2024-07-25 11:09'}, 'current': {'last_updated_epoch': 1721930400, 'last_updated': '2024-07-25 11:00', 'temp_c': 18.5, 'temp_f': 65.3, 'is_day': 1, 'condition': {'text': 'Patchy rain nearby', 'icon': '//cdn.weatherapi.com/weather/64x64/day/176.png', 'code': 1063}, 'wind_mph': 7.2, 'wind_kph': 11.5, 'wind_degree': 245, 'wind_dir': 'WSW', 'pressure_mb': 1018.0, 'pressure_in': 30.07, 'precip_mm': 0.01, 'precip_in': 0.0, 'humidity': 65, 'cloud': 72, 'feelslike_c': 18.5, 'feelslike_f': 65.3, 'windchill_c': 18.5, 'windchill_f': 65.3, 'heatindex_c': 18.5, 'heatindex_f': 65.3, 'dewpoint_c': 11.9, 'dewpoint_f': 53.4, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 4.0, 'gust_mph': 9.5, 'gust_kph': 15.3}}"}, {'url': 'https://www.wunderground.com/ho

In [5]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")

 ········


In [6]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

In [7]:
model_with_tools = model.bind_tools(tools)

In [8]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [10]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_GKKWZG91Q3CUgsHNXPiaHxZz', 'type': 'tool_call'}]


## Create the agent

In [11]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

## Run the agent

### let's see how it responds when there's no need to call a tool:

In [12]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='4b64b419-349f-44a8-aba5-9079d3dc1360'),
 AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 83, 'total_tokens': 93}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-573bc61e-3507-483b-9956-5382fcce8596-0', usage_metadata={'input_tokens': 83, 'output_tokens': 10, 'total_tokens': 93})]

### Let's now try it out on an example where it should be invoking the tool

In [14]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', id='31a1bcbf-ad08-4909-a443-08e97c487141'),
 AIMessage(content='I\'m sorry for the inconvenience. As an AI model developed by OpenAI, I currently don\'t have the capability to provide real-time information or updates such as weather forecasts. However, you can easily find the current weather in San Francisco by searching for "San Francisco weather" on any reliable weather forecasting website or app such as the National Weather Service, Weather.com, or your smartphone\'s weather app.', response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 88, 'total_tokens': 169}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0fdb27df-a579-4519-8b9d-621da55f625a-0', usage_metadata={'input_tokens': 88, 'output_tokens': 81, 'total_tokens': 169})]

## Streaming Messages

In [15]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_DUtYIGUEeJMaYbHTvb6YKqY8', 'function': {'arguments': '{\n  "query": "current weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 88, 'total_tokens': 111}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-33d25c6b-ab66-4da8-8d8a-2cb2f019ac02-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_DUtYIGUEeJMaYbHTvb6YKqY8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 88, 'output_tokens': 23, 'total_tokens': 111})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37

## Streaming tokens

In [16]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

D:\MyProjects\LangChain\langchain_conda_env\lib\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


--
Starting tool: tavily_search_results_json with inputs: {'query': 'current weather in San Francisco'}
Done tool: tavily_search_results_json
Tool output was: content=[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1721933207, 'localtime': '2024-07-25 11:46'}, 'current': {'last_updated_epoch': 1721933100, 'last_updated': '2024-07-25 11:45', 'temp_c': 18.5, 'temp_f': 65.3, 'is_day': 1, 'condition': {'text': 'Patchy rain nearby', 'icon': '//cdn.weatherapi.com/weather/64x64/day/176.png', 'code': 1063}, 'wind_mph': 7.2, 'wind_kph': 11.5, 'wind_degree': 245, 'wind_dir': 'WSW', 'pressure_mb': 1018.0, 'pressure_in': 30.07, 'precip_mm': 0.01, 'precip_in': 0.0, 'humidity': 65, 'cloud': 72, 'feelslike_c': 18.5, 'feelslike_f': 65.3, 'windchill_c': 18.5, 'windchill_f': 65.3, 'heatindex_c': 18.5, 'heatindex_f': 65.

## Adding in memory

In [17]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [18]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [19]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello, Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 85, 'total_tokens': 97}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d88c2d4f-90fe-40c9-bba4-4c938c76e550-0', usage_metadata={'input_tokens': 85, 'output_tokens': 12, 'total_tokens': 97})]}}
----


In [20]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob.', response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 109, 'total_tokens': 115}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-60be9105-991c-42b1-b9be-e741505a34e5-0', usage_metadata={'input_tokens': 109, 'output_tokens': 6, 'total_tokens': 115})]}}
----


#### If I want to start a new conversation, all I have to do is change the thread_id used

In [21]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="As an AI, I'm sorry but I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality.", response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 86, 'total_tokens': 129}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6a683e2f-170c-4a74-8fef-1eb227adadf9-0', usage_metadata={'input_tokens': 86, 'output_tokens': 43, 'total_tokens': 129})]}}
----
